In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import nltk
nltk.download('stopwords')
import gensim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
dataset = pd.read_csv('Corpus_não_processado2.csv', sep=';', header = None, encoding='latin-1')
dataset.head()

,0,1
0,1,Anti-tumor-promotion by principles obtained fr...
1,1,Effects of catechins on the mouse lung carcino...
2,1,Evaluation of cyclooxygenase-2 inhibitor for p...
3,1,Phase I study of E7010. E7010 is a novel sulfo...
4,1,"Studies on retinoids. IV. Design, synthesis, s..."


In [39]:
dataset.rename(columns={0:'Classe', 1:'Abstract'}, inplace=True)
dataset.head()

,Classe,Abstract
0,1,Anti-tumor-promotion by principles obtained fr...
1,1,Effects of catechins on the mouse lung carcino...
2,1,Evaluation of cyclooxygenase-2 inhibitor for p...
3,1,Phase I study of E7010. E7010 is a novel sulfo...
4,1,"Studies on retinoids. IV. Design, synthesis, s..."


In [40]:
dataset.loc[0, 'Abstract']

'Anti-tumor-promotion by principles obtained from Angelica keiskei. Potent anti-tumor promoter activity has been found in the nonpolar extracts of the root of Ashita-Ba, Angelica keiskei Koidz. (Umbelliferae), which is eaten as a vegetable in Japan. From this active fraction, two angular furanocoumarins, archangelicin (1), 8(S),9(R 9-angeloyloxy-8,9-dihydrooroselol (2), three linear furanocoumarins, psoralen (3), bergapten (4), xanthotoxin (5),, three chalcones, 4-hydroxyderricin (6), xanthoangelol (7), a novel chalcone named ashitaba-chalcone (8), were isolated. Among these compounds, two angular type furanocoumarins, 1, 2,, three chalcones, 6-8, suppressed 12-O-tetradecanoylphorbol-13-acetate (TPA stimulated 32Pi-incorporation into phospholipids of cultured cells, whereas coumarins 3-5 were less effective. In addition, chalcones 6, 7 were proved to have anti-tumor-promoting activity in mouse skin carcinogenesis induced by 7,12-dimethylbenz a anthracene (DMBA) plus TPA. Since chalcone

In [41]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in dataset['Abstract'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


(400, 0)

In [42]:
pd.isnull(docs_vectors).sum().sum()

0.0

In [43]:
docs_vectors['Classe'] = dataset['Classe']
docs_vectors = docs_vectors.dropna()

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

train_x, test_x, train_y, test_y = train_test_split(docs_vectors.drop('Classe', axis = 1),
                                                   docs_vectors['Classe'],
                                                   test_size = 0.2,
                                                   random_state = 1)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((320, 0), (320,), (80, 0), (80,))

In [45]:
model = AdaBoostClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)
from sklearn.metrics import accuracy_score
accuracy_score(test_y, test_pred)

ValueError: ignored